In [ ]:
#install spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#penamaan spark jadi "SPAM_EMAIL"
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SPAM_EMAIL').getOrCreate()

# Importt Data

In [ ]:
#memanggil data set atau import dataset 
path = '/content/spam.csv'

In [ ]:
data_original = spark.read.csv(path, inferSchema=True,header=True)

In [ ]:
#print or display dari data frame, ini hanya menampilkan nama kolum dan tipe data.
data_original.printSchema()

root
 |-- v1: string (nullable = true)
 |-- v2: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [ ]:
for keys,item in zip(data_original.head(5)[3].asDict().keys(),
                     data_original.head(5)[3].asDict().values()):
  print(keys+' : '+str(item))

v1 : ham
v2 : U dun say so early hor... U c already then say...
_c2 : None
_c3 : None
_c4 : None


In [ ]:
#menampilkan kolom kosong
from pyspark.sql.functions import col,isnan, when, count

data_original.select(
    [
        count(
            when(
                isnan(c) | col(c).isNull(), c
                 )
            ).alias(c) for c in data_original.columns
     ]
     ).show()

+---+---+----+----+----+
| v1| v2| _c2| _c3| _c4|
+---+---+----+----+----+
|  0|  1|5525|5562|5568|
+---+---+----+----+----+



In [ ]:
#untuk mengukur panjang kolom dari suatu data
from pyspark.sql.functions import length,mean

data = data_original.select('v1','v2')\
        .withColumnRenamed('v1','class')\
        .withColumn('length',length(data_original['v2']))\
        .withColumnRenamed('v2','message')\
        .na.drop()\
        .filter(
                (data_original['v1'] == 'ham') |\
                (data_original['v1'] == 'spam')
                )
#menghapus data kosong.

In [ ]:
data.show()

+-----+--------------------+------+
|class|             message|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if that��s t...|    58|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [ ]:
#menggabungkan data dan menhitung rata-rata
data.groupBy('class')\
    .agg(
        count('class'),
         mean('length')
         ).show()

+-----+------------+------------------+
|class|count(class)|       avg(length)|
+-----+------------+------------------+
|  ham|        4825| 71.07357512953368|
| spam|         747|138.45917001338688|
+-----+------------+------------------+



# SPARK MODELING

In [ ]:
#import StringIndexer untuk merubah tipe menjadi numrical.
from pyspark.ml.feature import (RegexTokenizer,StopWordsRemover, 
                                CountVectorizer,IDF,
                                StringIndexer,VectorAssembler)

In [ ]:
#penggunaan string indexer untuk merubah data
indexer = StringIndexer(
                        inputCol="class",
                        outputCol="label",
                        stringOrderType="frequencyDesc"
                        )

index_data = indexer.fit(data).transform(data)

index_data.groupBy('label')\
    .agg(
        count('label'),
         mean('length')
         ).show()

+-----+------------+------------------+
|label|count(label)|       avg(length)|
+-----+------------+------------------+
|  0.0|        4825| 71.07357512953368|
|  1.0|         747|138.45917001338688|
+-----+------------+------------------+



In [ ]:
#merubah data col Message ke col words. 
#Spasi atau space harus di hapuskan dan kalimat dipisahkan perkata
token = RegexTokenizer(
                        inputCol="message",
                        outputCol="words",
                        pattern="\\W"
                        )
token_data = token.transform(index_data)

token_data.show(5)

+-----+--------------------+------+-----+--------------------+
|class|             message|length|label|               words|
+-----+--------------------+------+-----+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar, joking,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|
|  ham|Nah I don't think...|    61|  0.0|[nah, i, don, t, ...|
+-----+--------------------+------+-----+--------------------+
only showing top 5 rows



In [ ]:
#kata yang tidak terdapat informasi akan di hapus oleh spark
stop = StopWordsRemover(inputCol='words', outputCol='n_words')

stop_data = stop.transform(token_data)

stop_data.select('words','n_words').show(5)

+--------------------+--------------------+
|               words|             n_words|
+--------------------+--------------------+
|[go, until, juron...|[go, jurong, poin...|
|[ok, lar, joking,...|[ok, lar, joking,...|
|[free, entry, in,...|[free, entry, 2, ...|
|[u, dun, say, so,...|[u, dun, say, ear...|
|[nah, i, don, t, ...|[nah, think, goes...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#membuat dictionary numeric dari semua isi pesan dengan CountVectorizer
countvec = CountVectorizer(inputCol='n_words',outputCol='cvec_words')

countvec_data = countvec.fit(stop_data).transform(stop_data)

countvec_data.select('cvec_words').show(1,truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------+
|cvec_words                                                                                                                                |
+------------------------------------------------------------------------------------------------------------------------------------------+
|(8537,[11,16,37,62,69,79,249,544,632,770,1252,1295,1308,2873,4668,4907],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [ ]:
#statistik numerik untuk mencari kata yang penting
idf_model = IDF(inputCol='cvec_words',outputCol='idf_words')

idf_data = idf_model.fit(countvec_data).transform(countvec_data)

idf_data.select('idf_words').show(1,truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|idf_words                                                                                                                                                                                                                                                                                                                                                                     |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#pemisahan vector antara kolom features dan label 
assembler = VectorAssembler(inputCols=['idf_words','length'],outputCol='features')

treated_data = assembler.transform(idf_data)

treated_data.select('features','label').show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(8538,[11,16,37,6...|  0.0|
|(8538,[0,9,244,36...|  0.0|
|(8538,[2,10,23,24...|  1.0|
|(8538,[0,57,85,86...|  0.0|
|(8538,[53,136,366...|  0.0|
|(8538,[9,15,21,26...|  1.0|
|(8538,[15,132,286...|  0.0|
|(8538,[149,157,31...|  0.0|
|(8538,[1,64,82,14...|  1.0|
|(8538,[0,1,10,31,...|  1.0|
|(8538,[3,22,29,33...|  0.0|
|(8538,[6,17,21,24...|  1.0|
|(8538,[10,24,26,5...|  1.0|
|(8538,[45,77,84,1...|  0.0|
|(8538,[479,677,85...|  0.0|
|(8538,[24,37,80,1...|  1.0|
|(8538,[3,41,63,27...|  0.0|
|(8538,[0,2,71,73,...|  0.0|
|(8538,[0,72,91,13...|  0.0|
|(8538,[5,24,26,42...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [ ]:
treated_data.select('features')

DataFrame[features: vector]

# Pipeline

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipe_prep_data = Pipeline(stages=[indexer, token, stop, countvec,
                                  idf_model, assembler])

In [ ]:
prep_data = pipe_prep_data.fit(data).transform(data)

In [ ]:
prep_data.select('features','label').show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(8538,[11,16,37,6...|  0.0|
|(8538,[0,9,244,36...|  0.0|
|(8538,[2,10,23,24...|  1.0|
|(8538,[0,57,85,86...|  0.0|
|(8538,[53,136,366...|  0.0|
|(8538,[9,15,21,26...|  1.0|
|(8538,[15,132,286...|  0.0|
|(8538,[149,157,31...|  0.0|
|(8538,[1,64,82,14...|  1.0|
|(8538,[0,1,10,31,...|  1.0|
|(8538,[3,22,29,33...|  0.0|
|(8538,[6,17,21,24...|  1.0|
|(8538,[10,24,26,5...|  1.0|
|(8538,[45,77,84,1...|  0.0|
|(8538,[479,677,85...|  0.0|
|(8538,[24,37,80,1...|  1.0|
|(8538,[3,41,63,27...|  0.0|
|(8538,[0,2,71,73,...|  0.0|
|(8538,[0,72,91,13...|  0.0|
|(8538,[5,24,26,42...|  1.0|
+--------------------+-----+
only showing top 20 rows



# Machine Learning Modelling

# Train and Test
membagi data menjadi data train dan data test

In [ ]:
#membagi data dengan random state nya 0.8 dan 0.2.
#0.8 untuk train dan 0.2 untuk test
train_data,test_data = prep_data.randomSplit([0.8,0.2],seed = 1)

In [ ]:
train_data.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 3832|
|  1.0|  594|
+-----+-----+



In [ ]:
test_data.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  993|
|  1.0|  153|
+-----+-----+



# Naive Bayes

In [ ]:
#import algoritma Naive Bayes
from pyspark.ml.classification import NaiveBayes

In [ ]:
nB = NaiveBayes(featuresCol='features',
                      labelCol='label',
                      modelType='multinomial')

In [ ]:
nB_model = nB.fit(train_data)

In [ ]:
test_results = nB_model.transform(test_data)
test_results.columns

['class',
 'message',
 'length',
 'label',
 'words',
 'n_words',
 'cvec_words',
 'idf_words',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [ ]:
test_results.select('label','prediction','rawPrediction','probability').show()

+-----+----------+--------------------+--------------------+
|label|prediction|       rawPrediction|         probability|
+-----+----------+--------------------+--------------------+
|  0.0|       0.0|[-859.65131284924...|[1.0,4.3097634138...|
|  0.0|       0.0|[-4650.7280415482...|[1.0,7.7828248558...|
|  0.0|       0.0|[-4650.7280415482...|[1.0,7.7828248558...|
|  0.0|       0.0|[-1203.6600955646...|[1.0,1.2878081462...|
|  0.0|       0.0|[-1411.3269760801...|[1.0,1.3678122809...|
|  0.0|       0.0|[-1220.8401401176...|[1.0,1.3227880678...|
|  0.0|       0.0|[-480.14388362531...|[1.0,6.1770067477...|
|  0.0|       0.0|[-292.17748613866...|[1.0,1.3484351044...|
|  0.0|       0.0|[-426.76108379898...|[1.0,3.6604115341...|
|  0.0|       0.0|[-1905.6818024263...|[1.0,3.6097431012...|
|  0.0|       0.0|[-1674.1179745284...|[0.95938665094168...|
|  0.0|       0.0|[-227.60994623319...|[0.99999978380961...|
|  0.0|       0.0|[-463.80407128968...|[1.0,1.8051991557...|
|  0.0|       0.0|[-376.

In [ ]:
#Evaluation Model dengan Confusion Matrix
def ConfusionMatrixStats(test_results): 

    #True Positives
    TP = test_results.filter((test_results['prediction']==1) & (test_results['label'] == 1)).count()
    #True Negatives
    TN = test_results.filter((test_results['prediction']==0) & (test_results['label'] == 0)).count()
    #False Positives
    FP = test_results.filter((test_results['prediction']==1) & (test_results['label'] == 0)).count()
    #False Negatives
    FN = test_results.filter((test_results['prediction']==0) & (test_results['label'] == 1)).count()

    mat = [[TP,FN],[FP,TN]]
    print('-----------------------------------------------')
    print('TP  =   {}         FN  =   {}   '.format(TP,FN))
    print('FP  =   {}         TN  =   {}   '.format(FP,TN))

    accuracy = (TP+TN)/(TP+TN+FP+FN)
    print('The accuracy for the test dataset is {}.'.format(accuracy))
    #recall or true positive rate
    recall = TP/(TP+FN)
    print('The recall for the test dataset is {}.'.format(recall))
    #negative recall or true negative rate
    selec = TN/(FP+TN)
    print('The selectivity for the test dataset is {}.'.format(selec))
    #precision or positive predicted value
    prec = TP/(FP+TP)
    print('The precision for the test dataset is {}.'.format(prec))


In [ ]:
#hasil confusion matrix dari Naive Bayes
ConfusionMatrixStats(test_results)

-----------------------------------------------
TP  =   151         FN  =   2   
FP  =   49         TN  =   944   
The accuracy for the test dataset is 0.9554973821989529.
The recall for the test dataset is 0.9869281045751634.
The selectivity for the test dataset is 0.9506545820745217.
The precision for the test dataset is 0.755.


#Random Forest 

In [ ]:
#import algoritma Random forest
from pyspark.ml.classification import RandomForestClassifier as RFC

In [ ]:
#hasil confusiion matrix dengan algoritma Random Forest
rfc_model = RFC(
                featuresCol='features',
                labelCol='label',
                maxDepth=10,
                numTrees=200,
                impurity='entropy')

rfc_test_results = rfc_model.fit(train_data).transform(test_data)
ConfusionMatrixStats(rfc_test_results)

-----------------------------------------------
TP  =   25         FN  =   128   
FP  =   0         TN  =   993   
The accuracy for the test dataset is 0.8883071553228621.
The recall for the test dataset is 0.16339869281045752.
The selectivity for the test dataset is 1.0.
The precision for the test dataset is 1.0.


# Kesimpulan
Model Evaluasi dengan Random Forest lebih rendah dibandingkan Naive Bayes. Jadi penggunaan Naive Bayes akurasinya lebih bagus dibandingkan Random Forest.